# Pembuatan Model machine Learning

## Import Libry yang di gunakan

In [7]:
! pip install gensim

In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import tensorflow as tf
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras import regularizers
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
# Import library tambahan (ML klasik + Deep Learning)
import warnings
warnings.filterwarnings('ignore')

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

from gensim.models import Word2Vec
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
# Fungsi balancing: kombinasi oversampling + undersampling
from scipy.sparse import issparse
from collections import Counter

## Load Data

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df = pd.read_csv('../content/drive/MyDrive/WILDAN FADHIL NAZARUDDIN (PERSONAL)/PROJECK/MACHINE LEARNING/hasil_pelabelan (1).csv')

In [10]:
df.head()

,content,score,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
0,"aplikasi nya bagus, aku sering cek out apa apa...",3,aplikasi nya bagus aku sering cek out apa apa ...,aplikasi nya bagus aku sering cek out apa apa ...,aplikasi nya bagus aku sering cek out apa apa ...,"['aplikasi', 'nya', 'bagus', 'aku', 'sering', ...","['aplikasi', 'bagus', 'cek', 'iklannya', 'kond...",aplikasi bagus cek iklannya kondisikan kdang b...,-14,negative
1,Bagus sering banyak diskon atau harga harga ya...,4,Bagus sering banyak diskon atau harga harga ya...,bagus sering banyak diskon atau harga harga ya...,bagus sering banyak diskon atau harga harga ya...,"['bagus', 'sering', 'banyak', 'diskon', 'atau'...","['bagus', 'diskon', 'harga', 'harga', 'anjlok'...",bagus diskon harga harga anjlok terkadang suka...,-12,negative
2,"Apk yg bagus,,memesan brg & lain sebagiannya j...",5,Apk yg bagusmemesan brg lain sebagiannya jadi...,apk yg bagusmemesan brg lain sebagiannya jadi...,aplikasi yang bagusmemesan barang lain sebagia...,"['aplikasi', 'yang', 'bagusmemesan', 'barang',...","['aplikasi', 'bagusmemesan', 'barang', 'sebagi...",aplikasi bagusmemesan barang sebagiannya mudah...,5,positive
3,"aplikasi aneh, akun SDH gold bahkan platinum, ...",1,aplikasi aneh akun SDH gold bahkan platinum ka...,aplikasi aneh akun sdh gold bahkan platinum ka...,aplikasi aneh akun sudah gold bahkan platinum ...,"['aplikasi', 'aneh', 'akun', 'sudah', 'gold', ...","['aplikasi', 'aneh', 'akun', 'gold', 'platinum...",aplikasi aneh akun gold platinum diskon pas di...,-14,negative
4,"pertama nya bagus belanja via COD lancar, beba...",1,pertama nya bagus belanja via COD lancar bebas...,pertama nya bagus belanja via cod lancar bebas...,pertama nya bagus belanja via bayar ditempat l...,"['pertama', 'nya', 'bagus', 'belanja', 'via', ...","['bagus', 'belanja', 'via', 'bayar', 'ditempat...",bagus belanja via bayar ditempat lancar bebas ...,-2,negative


## Data Slpiting

In [11]:
df = df.dropna(subset=['text_akhir', 'polarity']).copy()
X = df['text_akhir'].astype(str)
y = df['polarity'].astype(str)

print('Jumlah data:', len(df))
print('Distribusi kelas data asli:')
print(y.value_counts())

Jumlah data: 71020
Distribusi kelas data asli:
polarity
positive    33833
negative    33070
neutral      4117
Name: count, dtype: int64


In [12]:
# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=5000, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

In [13]:
# Konversi hasil ekstraksi fitur menjadi dataframe
model_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [14]:
# Encode label agar bisa dipakai model sklearn dan TensorFlow
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

In [15]:
print("Mapping label:", dict(zip(encoder.classes_, encoder.transform(encoder.classes_))))

Mapping label: {'negative': np.int64(0), 'neutral': np.int64(1), 'positive': np.int64(2)}


In [16]:
# CELL BARU 2: UNDER-SAMPLING + evaluasi model tradisional
rus = RandomUnderSampler(sampling_strategy='not minority', random_state=42)
X_train_under, y_train_under = rus.fit_resample(X_train, y_train)

print("Distribusi y_train asli   :", Counter(y_train))
print("Distribusi y_train under  :", Counter(y_train_under))

Distribusi y_train asli   : Counter({np.int64(2): 27066, np.int64(0): 26456, np.int64(1): 3294})
Distribusi y_train under  : Counter({np.int64(0): 3294, np.int64(1): 3294, np.int64(2): 3294})


In [17]:
try:
    from xgboost import XGBClassifier
    XGBOOST_READY = True
except Exception:
    XGBOOST_READY = False
    print("xgboost belum terpasang. Di Colab jalankan: !pip install xgboost")

### Tahap 1 — Data Splitting Imbalance + ML Klasik Versi 2

Pada data 3 kelas (`positif`, `negatif`, `netral`), kita tambahkan pendekatan **oversampling + undersampling** di tahap data splitting agar distribusi kelas lebih seimbang.

Konfigurasi ML klasik versi 2:
1. Pelatihan: **SVM**, Ekstraksi Fitur: **TF-IDF**, Pembagian Data: **80/20**
2. Pelatihan: **Random Forest**, Ekstraksi Fitur: **Word2Vec**, Pembagian Data: **80/20**
3. Pelatihan: **Random Forest**, Ekstraksi Fitur: **TF-IDF**, Pembagian Data: **70/30**

In [18]:
# Ambil data dari variabel df yang sudah ada
df_v2 = df.copy()

# Deteksi kolom teks dan label secara aman
candidate_text_cols = ['text_akhir', 'text_clean', 'clean_text', 'text']
candidate_label_cols = ['polarity', 'sentiment', 'label']

text_col = next((c for c in candidate_text_cols if c in df_v2.columns), None)
label_col = next((c for c in candidate_label_cols if c in df_v2.columns), None)

if text_col is None or label_col is None:
    raise ValueError(f"Kolom tidak ditemukan. text_col={text_col}, label_col={label_col}")

df_v2 = df_v2.dropna(subset=[text_col, label_col]).copy()
df_v2[text_col] = df_v2[text_col].astype(str)

print(f"Kolom teks: {text_col}")
print(f"Kolom label: {label_col}")
print("Distribusi label awal:")
print(df_v2[label_col].value_counts())

Kolom teks: text_akhir
Kolom label: polarity
Distribusi label awal:
polarity
positive    33833
negative    33070
neutral      4117
Name: count, dtype: int64


In [27]:
# Tetap menggunakan df sebagai sumber utama
required_cols = ['text_akhir', 'polarity']
missing_cols = [c for c in required_cols if c not in df.columns]
if missing_cols:
    raise ValueError(f"Kolom wajib tidak ditemukan di df: {missing_cols}")

df_split = df.dropna(subset=required_cols).copy()
df_split['text_akhir'] = df_split['text_akhir'].astype(str)

# Encode label dari df (3 kelas)
encoder_v2 = LabelEncoder()
y_split = encoder_v2.fit_transform(df_split['polarity'])
X_text_split = df_split['text_akhir'].values

print("Mapping label:", dict(zip(encoder_v2.classes_, encoder_v2.transform(encoder_v2.classes_))))

def balance_over_under(X_train, y_train, random_state=42):
    ros = RandomOverSampler(random_state=random_state)
    rus = RandomUnderSampler(random_state=random_state)
    X_over, y_over = ros.fit_resample(X_train, y_train)
    X_bal, y_bal = rus.fit_resample(X_over, y_over)
    return X_bal, y_bal

# ===== Split 1: TF-IDF + 80/20 =====
X_tr_text_80, X_te_text_80, y_tr_80, y_te_80 = train_test_split(
    X_text_split, y_split, test_size=0.2, random_state=42, stratify=y_split
)

tfidf_80 = TfidfVectorizer(max_features=5000)
X_tr_tfidf_80 = tfidf_80.fit_transform(X_tr_text_80)
X_te_tfidf_80 = tfidf_80.transform(X_te_text_80)

X_tr_tfidf_80_bal, y_tr_tfidf_80_bal = balance_over_under(X_tr_tfidf_80, y_tr_80)

print("Distribusi kelas TF-IDF 80/20 sebelum balancing:", Counter(y_tr_80))
print("Distribusi kelas TF-IDF 80/20 sesudah balancing:", Counter(y_tr_tfidf_80_bal))

# ===== Split 2: Word2Vec + 80/20 =====
X_tr_text_w2v, X_te_text_w2v, y_tr_w2v, y_te_w2v = train_test_split(
    X_text_split, y_split, test_size=0.2, random_state=42, stratify=y_split
)

def simple_tokenize(text):
    return str(text).lower().split()

tokens_train = [simple_tokenize(t) for t in X_tr_text_w2v]
tokens_test = [simple_tokenize(t) for t in X_te_text_w2v]

w2v_model = Word2Vec(
    sentences=tokens_train,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    seed=42
)

def sent_vector(tokens, model, vector_size=100):
    valid_tokens = [tok for tok in tokens if tok in model.wv]
    if not valid_tokens:
        return np.zeros(vector_size, dtype='float32')
    return np.mean(model.wv[valid_tokens], axis=0)

X_tr_w2v_80 = np.array([sent_vector(t, w2v_model, 100) for t in tokens_train])
X_te_w2v_80 = np.array([sent_vector(t, w2v_model, 100) for t in tokens_test])

X_tr_w2v_80_bal, y_tr_w2v_80_bal = balance_over_under(X_tr_w2v_80, y_tr_w2v)

print("Distribusi kelas Word2Vec 80/20 sebelum balancing:", Counter(y_tr_w2v))
print("Distribusi kelas Word2Vec 80/20 sesudah balancing:", Counter(y_tr_w2v_80_bal))

# ===== Split 3: TF-IDF + 70/30 =====
X_tr_text_70, X_te_text_70, y_tr_70, y_te_70 = train_test_split(
    X_text_split, y_split, test_size=0.3, random_state=42, stratify=y_split
)

tfidf_70 = TfidfVectorizer(max_features=5000)
X_tr_tfidf_70 = tfidf_70.fit_transform(X_tr_text_70)
X_te_tfidf_70 = tfidf_70.transform(X_te_text_70)

X_tr_tfidf_70_bal, y_tr_tfidf_70_bal = balance_over_under(X_tr_tfidf_70, y_tr_70)

print("Distribusi kelas TF-IDF 70/30 sebelum balancing:", Counter(y_tr_70))
print("Distribusi kelas TF-IDF 70/30 sesudah balancing:", Counter(y_tr_tfidf_70_bal))

Mapping label: {'negative': np.int64(0), 'neutral': np.int64(1), 'positive': np.int64(2)}
Distribusi kelas TF-IDF 80/20 sebelum balancing: Counter({np.int64(2): 27066, np.int64(0): 26456, np.int64(1): 3294})
Distribusi kelas TF-IDF 80/20 sesudah balancing: Counter({np.int64(0): 27066, np.int64(1): 27066, np.int64(2): 27066})
Distribusi kelas Word2Vec 80/20 sebelum balancing: Counter({np.int64(2): 27066, np.int64(0): 26456, np.int64(1): 3294})
Distribusi kelas Word2Vec 80/20 sesudah balancing: Counter({np.int64(0): 27066, np.int64(1): 27066, np.int64(2): 27066})
Distribusi kelas TF-IDF 70/30 sebelum balancing: Counter({np.int64(2): 23683, np.int64(0): 23149, np.int64(1): 2882})
Distribusi kelas TF-IDF 70/30 sesudah balancing: Counter({np.int64(0): 23683, np.int64(1): 23683, np.int64(2): 23683})


## Tahap 2 — Data Splitting untuk Deep Learning

Kita gunakan pola pembagian data yang sama:
1. TF-IDF + 80/20
2. Word2Vec + 80/20
3. TF-IDF + 70/30

Data split deep learning diambil dari split yang sudah dibuat agar konsisten perbandingannya.

In [28]:
# Konversi fitur untuk model deep learning
X_tr_tfidf_80_dl = X_tr_tfidf_80_bal.astype('float32').toarray()
X_te_tfidf_80_dl = X_te_tfidf_80.astype('float32').toarray()
y_tr_tfidf_80_dl = y_tr_tfidf_80_bal
y_te_tfidf_80_dl = y_te_80

X_tr_w2v_80_dl = X_tr_w2v_80_bal.astype('float32')
X_te_w2v_80_dl = X_te_w2v_80.astype('float32')
y_tr_w2v_80_dl = y_tr_w2v_80_bal
y_te_w2v_80_dl = y_te_w2v

X_tr_tfidf_70_dl = X_tr_tfidf_70_bal.astype('float32').toarray()
X_te_tfidf_70_dl = X_te_tfidf_70.astype('float32').toarray()
y_tr_tfidf_70_dl = y_tr_tfidf_70_bal
y_te_tfidf_70_dl = y_te_70

num_classes_v2 = len(encoder_v2.classes_)

### Membuat Model Machine Learning

#### Membuat model dengan algoritma Naive Bayes

In [29]:
# Naive Bayes
naive_bayes_under = BernoulliNB()
naive_bayes_under.fit(X_train_under.toarray(), y_train_under)
y_pred_train_nb_under = naive_bayes_under.predict(X_train_under.toarray())
y_pred_test_nb_under = naive_bayes_under.predict(X_test.toarray())
accuracy_train_nb_under = accuracy_score(y_train_under, y_pred_train_nb_under)
accuracy_test_nb_under = accuracy_score(y_test, y_pred_test_nb_under)

In [30]:
# Logistic Regression
logistic_regression_under = LogisticRegression(max_iter=2000, random_state=42)
logistic_regression_under.fit(X_train_under.toarray(), y_train_under)
y_pred_train_lr_under = logistic_regression_under.predict(X_train_under.toarray())
y_pred_test_lr_under = logistic_regression_under.predict(X_test.toarray())
accuracy_train_lr_under = accuracy_score(y_train_under, y_pred_train_lr_under)
accuracy_test_lr_under = accuracy_score(y_test, y_pred_test_lr_under)

In [31]:
# Decision Tree
decision_tree_under = DecisionTreeClassifier(random_state=42)
decision_tree_under.fit(X_train_under.toarray(), y_train_under)
y_pred_train_dt_under = decision_tree_under.predict(X_train_under.toarray())
y_pred_test_dt_under = decision_tree_under.predict(X_test.toarray())
accuracy_train_dt_under = accuracy_score(y_train_under, y_pred_train_dt_under)
accuracy_test_dt_under = accuracy_score(y_test, y_pred_test_dt_under)

In [32]:
results_under = pd.DataFrame({
    "Model": ["Naive Bayes", "Logistic Regression", "Decision Tree"],
    "Accuracy Train (Under)": [accuracy_train_nb_under, accuracy_train_lr_under, accuracy_train_dt_under],
    "Accuracy Test (Under)": [accuracy_test_nb_under, accuracy_test_lr_under, accuracy_test_dt_under]
}).sort_values(by="Accuracy Test (Under)", ascending=False)

print(results_under)

                 Model  Accuracy Train (Under)  Accuracy Test (Under)
1  Logistic Regression                0.849018               0.745283
2        Decision Tree                0.999899               0.577936
0          Naive Bayes                0.655131               0.542875


### Model Klasik 1 — SVM + TF-IDF (80/20)

In [33]:
svm_tfidf_80 = SVC(kernel='linear', C=1.0, random_state=42)
svm_tfidf_80.fit(X_tr_tfidf_80_bal, y_tr_tfidf_80_bal)

pred_svm_80 = svm_tfidf_80.predict(X_te_tfidf_80)
acc_svm_80 = accuracy_score(y_te_80, pred_svm_80)

print(f"Akurasi SVM + TF-IDF (80/20): {acc_svm_80:.4f}")
print(classification_report(y_te_80, pred_svm_80, target_names=encoder_v2.classes_))

Akurasi SVM + TF-IDF (80/20): 0.8543
              precision    recall  f1-score   support

    negative       0.96      0.86      0.91      6614
     neutral       0.26      0.70      0.38       823
    positive       0.96      0.86      0.91      6767

    accuracy                           0.85     14204
   macro avg       0.73      0.81      0.73     14204
weighted avg       0.92      0.85      0.88     14204



### Model Klasik 2 — Random Forest + Word2Vec (80/20)

In [34]:
rf_w2v_80 = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced_subsample'
)
rf_w2v_80.fit(X_tr_w2v_80_bal, y_tr_w2v_80_bal)

pred_rf_w2v_80 = rf_w2v_80.predict(X_te_w2v_80)
acc_rf_w2v_80 = accuracy_score(y_te_w2v, pred_rf_w2v_80)

print(f"Akurasi RF + Word2Vec (80/20): {acc_rf_w2v_80:.4f}")
print(classification_report(y_te_w2v, pred_rf_w2v_80, target_names=encoder_v2.classes_))

Akurasi RF + Word2Vec (80/20): 0.7289
              precision    recall  f1-score   support

    negative       0.71      0.80      0.75      6614
     neutral       0.84      0.08      0.15       823
    positive       0.75      0.74      0.74      6767

    accuracy                           0.73     14204
   macro avg       0.77      0.54      0.55     14204
weighted avg       0.74      0.73      0.71     14204



### Model Klasik 3 — Random Forest + TF-IDF (70/30)

In [35]:
### Model Klasik 3 — Random Forest + TF-IDF (70/30)
rf_tfidf_70 = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced_subsample'
)
rf_tfidf_70.fit(X_tr_tfidf_70_bal, y_tr_tfidf_70_bal)

pred_rf_tfidf_70 = rf_tfidf_70.predict(X_te_tfidf_70)
acc_rf_tfidf_70 = accuracy_score(y_te_70, pred_rf_tfidf_70)

print(f"Akurasi RF + TF-IDF (70/30): {acc_rf_tfidf_70:.4f}")
print(classification_report(y_te_70, pred_rf_tfidf_70, target_names=encoder_v2.classes_))

Akurasi RF + TF-IDF (70/30): 0.8110
              precision    recall  f1-score   support

    negative       0.80      0.86      0.83      9921
     neutral       0.64      0.23      0.34      1235
    positive       0.83      0.83      0.83     10150

    accuracy                           0.81     21306
   macro avg       0.76      0.64      0.67     21306
weighted avg       0.81      0.81      0.80     21306



In [47]:
# Ringkasan model klasik (baseline, tanpa tuning)
classic_results = pd.DataFrame([
    {"Model": "SVM + TF-IDF (80/20)", "Accuracy": acc_svm_80, "kind": "svm_tfidf80"},
    {"Model": "RF + Word2Vec (80/20)", "Accuracy": acc_rf_w2v_80, "kind": "rf_w2v80"},
    {"Model": "RF + TF-IDF (70/30)", "Accuracy": acc_rf_tfidf_70, "kind": "rf_tfidf70"}
]).sort_values("Accuracy", ascending=False).reset_index(drop=True)

print("Ringkasan akurasi model klasik (baseline):")
display(classic_results[["Model", "Accuracy"]])

best_classic_name = classic_results.loc[0, "Model"]
best_classic_kind = classic_results.loc[0, "kind"]
best_classic_acc = float(classic_results.loc[0, "Accuracy"])

# Simpan model klasik terbaik baseline untuk perbandingan final
if best_classic_kind == "svm_tfidf80":
    best_classic_model = svm_tfidf_80
elif best_classic_kind == "rf_w2v80":
    best_classic_model = rf_w2v_80
else:
    best_classic_model = rf_tfidf_70

print(f"Model klasik terbaik (baseline): {best_classic_name} | akurasi={best_classic_acc:.4f}")

Ringkasan akurasi model klasik (baseline):


,Model,Accuracy
0,SVM + TF-IDF (80/20),0.854266
1,RF + TF-IDF (70/30),0.811039
2,RF + Word2Vec (80/20),0.728879


Model klasik terbaik (baseline): SVM + TF-IDF (80/20) | akurasi=0.8543


## Membuat Deep Learning

In [37]:
def build_mlp(input_dim, num_classes):
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

early_stop_v2 = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

### Deep Learning Model 1 — MLP + TF-IDF (80/20)

In [38]:
dl_mlp_tfidf_80 = build_mlp(X_tr_tfidf_80_dl.shape[1], num_classes_v2)

history_dl1 = dl_mlp_tfidf_80.fit(
    X_tr_tfidf_80_dl, y_tr_tfidf_80_dl,
    validation_split=0.2,
    epochs=12,
    batch_size=128,
    callbacks=[early_stop_v2],
    verbose=1
)

_, acc_dl1 = dl_mlp_tfidf_80.evaluate(X_te_tfidf_80_dl, y_te_tfidf_80_dl, verbose=0)
print(f"Akurasi DL-1 (MLP + TF-IDF 80/20): {acc_dl1:.4f}")

Epoch 1/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.6882 - loss: 0.6903 - val_accuracy: 0.8593 - val_loss: 0.3835
Epoch 2/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9628 - loss: 0.1189 - val_accuracy: 0.9042 - val_loss: 0.2860
Epoch 3/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9882 - loss: 0.0433 - val_accuracy: 0.8996 - val_loss: 0.3481
Epoch 4/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9943 - loss: 0.0208 - val_accuracy: 0.8964 - val_loss: 0.4059
Epoch 5/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9962 - loss: 0.0144 - val_accuracy: 0.9034 - val_loss: 0.4114
Akurasi DL-1 (MLP + TF-IDF 80/20): 0.8959


### Deep Learning Model 2 — MLP + Word2Vec (80/20)

In [39]:
dl_mlp_w2v_80 = build_mlp(X_tr_w2v_80_dl.shape[1], num_classes_v2)

history_dl2 = dl_mlp_w2v_80.fit(
    X_tr_w2v_80_dl, y_tr_w2v_80_dl,
    validation_split=0.2,
    epochs=12,
    batch_size=128,
    callbacks=[early_stop_v2],
    verbose=1
)

_, acc_dl2 = dl_mlp_w2v_80.evaluate(X_te_w2v_80_dl, y_te_w2v_80_dl, verbose=0)
print(f"Akurasi DL-2 (MLP + Word2Vec 80/20): {acc_dl2:.4f}")

Epoch 1/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.5462 - loss: 0.9277 - val_accuracy: 0.3192 - val_loss: 1.2739
Epoch 2/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6055 - loss: 0.8362 - val_accuracy: 0.2300 - val_loss: 1.5323
Epoch 3/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6266 - loss: 0.8043 - val_accuracy: 0.3491 - val_loss: 1.2870
Epoch 4/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6350 - loss: 0.7901 - val_accuracy: 0.3755 - val_loss: 1.2537
Epoch 5/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6458 - loss: 0.7715 - val_accuracy: 0.3743 - val_loss: 1.2843
Epoch 6/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6571 - loss: 0.7621 - val_accuracy: 0.4794 - val_loss: 1.0379
Epoch 7/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6661 - loss: 0.7409 - val_accuracy: 0.4581 - val_loss: 1.1262
Epoch 8/12
508/508 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6716 - loss: 0.7309 - val_accuracy: 0.

### Deep Learning Model 3 — MLP + TF-IDF (70/30)

In [40]:
dl_mlp_tfidf_70 = build_mlp(X_tr_tfidf_70_dl.shape[1], num_classes_v2)

history_dl3 = dl_mlp_tfidf_70.fit(
    X_tr_tfidf_70_dl, y_tr_tfidf_70_dl,
    validation_split=0.2,
    epochs=12,
    batch_size=128,
    callbacks=[early_stop_v2],
    verbose=1
)

_, acc_dl3 = dl_mlp_tfidf_70.evaluate(X_te_tfidf_70_dl, y_te_tfidf_70_dl, verbose=0)
print(f"Akurasi DL-3 (MLP + TF-IDF 70/30): {acc_dl3:.4f}")

Epoch 1/12
445/445 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.6649 - loss: 0.7186 - val_accuracy: 0.8274 - val_loss: 0.4410
Epoch 2/12
445/445 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9613 - loss: 0.1270 - val_accuracy: 0.8814 - val_loss: 0.3361
Epoch 3/12
445/445 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9859 - loss: 0.0502 - val_accuracy: 0.8944 - val_loss: 0.3463
Epoch 4/12
445/445 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9937 - loss: 0.0237 - val_accuracy: 0.8900 - val_loss: 0.4099
Epoch 5/12
445/445 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9960 - loss: 0.0154 - val_accuracy: 0.8883 - val_loss: 0.4815
Akurasi DL-3 (MLP + TF-IDF 70/30): 0.8890


### Deep Learning Model 4 — XGBoost

In [44]:
acc_xgb = np.nan
xgb_model = None

if XGBOOST_READY:
    xgb_model = XGBClassifier(
        objective='multi:softprob',
        num_class=num_classes_v2,
        eval_metric='mlogloss',
        n_estimators=300,
        learning_rate=0.08,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42,
        n_jobs=-1
    )
    # Fix: Use the balanced feature set X_tr_tfidf_80_bal
    xgb_model.fit(X_tr_tfidf_80_bal, y_tr_tfidf_80_bal)
    pred_xgb = xgb_model.predict(X_te_tfidf_80)
    acc_xgb = accuracy_score(y_te_80, pred_xgb)
    print(f"Akurasi XGBoost: {acc_xgb:.4f}")
    print(classification_report(y_te_80, pred_xgb, target_names=encoder_v2.classes_))
else:
    print("XGBoost dilewati karena paket belum tersedia.")

Akurasi XGBoost: 0.7473
              precision    recall  f1-score   support

    negative       0.85      0.77      0.81      6614
     neutral       0.17      0.47      0.25       823
    positive       0.87      0.76      0.81      6767

    accuracy                           0.75     14204
   macro avg       0.63      0.67      0.62     14204
weighted avg       0.82      0.75      0.78     14204



### Evaluasi Deep Learning + Pemilihan Akurasi Tertinggi

In [45]:
deep_results = pd.DataFrame([
    {"Model": "DL-1 MLP + TF-IDF (80/20)", "Accuracy": float(acc_dl1), "kind": "dl1"},
    {"Model": "DL-2 MLP + Word2Vec (80/20)", "Accuracy": float(acc_dl2), "kind": "dl2"},
    {"Model": "DL-3 MLP + TF-IDF (70/30)", "Accuracy": float(acc_dl3), "kind": "dl3"},
    {"Model": "XGBoost", "Accuracy": float(acc_xgb) if not np.isnan(acc_xgb) else -1.0, "kind": "xgb"}
]).sort_values("Accuracy", ascending=False).reset_index(drop=True)

print("Ringkasan akurasi model deep learning/xgboost:")
display(deep_results[['Model', 'Accuracy']])

best_deep_kind = deep_results.loc[0, 'kind']
best_deep_name = deep_results.loc[0, 'Model']
print(f"Model deep terbaik: {best_deep_name}")

# Simpan model deploy deep terbaik
if best_deep_kind == 'dl1':
    deployed_deep_model = dl_mlp_tfidf_80
    deep_feature_type = 'tfidf80'
elif best_deep_kind == 'dl2':
    deployed_deep_model = dl_mlp_w2v_80
    deep_feature_type = 'w2v80'
elif best_deep_kind == 'dl3':
    deployed_deep_model = dl_mlp_tfidf_70
    deep_feature_type = 'tfidf70'
else:
    deployed_deep_model = xgb_model
    deep_feature_type = 'xgb_tfidf80'

Ringkasan akurasi model deep learning/xgboost:


,Model,Accuracy
0,DL-1 MLP + TF-IDF (80/20),0.895945
1,DL-3 MLP + TF-IDF (70/30),0.889045
2,XGBoost,0.747254
3,DL-2 MLP + Word2Vec (80/20),0.576035


Model deep terbaik: DL-1 MLP + TF-IDF (80/20)


### Hyperparameter Tuning Deep Learning (Fokus Model Terbaik)

Berdasarkan evaluasi, model deep terbaik awal adalah **DL-1 MLP + TF-IDF (80/20)**.
Tahap ini melakukan tuning ringan pada arsitektur MLP (jumlah neuron, dropout, learning rate, dan batch size) lalu memilih konfigurasi dengan akurasi validasi terbaik.

In [48]:
# Tuning ringan untuk DL-1 (MLP + TF-IDF 80/20)
def build_mlp_tuned(input_dim, num_classes, units_1=256, units_2=128, dropout_1=0.3, dropout_2=0.2, lr=1e-3):
    model = Sequential([
        Dense(units_1, activation='relu', input_shape=(input_dim,)),
        Dropout(dropout_1),
        Dense(units_2, activation='relu'),
        Dropout(dropout_2),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

search_space = [
    {"units_1": 256, "units_2": 128, "dropout_1": 0.3, "dropout_2": 0.2, "lr": 1e-3, "batch_size": 128},
    {"units_1": 384, "units_2": 192, "dropout_1": 0.35, "dropout_2": 0.25, "lr": 8e-4, "batch_size": 128},
    {"units_1": 512, "units_2": 256, "dropout_1": 0.4, "dropout_2": 0.3, "lr": 7e-4, "batch_size": 256}
]

best_dl_tuned_model = None
best_dl_tuned_cfg = None
best_dl_tuned_val_acc = -1.0
best_dl_tuned_test_acc = -1.0

for cfg in search_space:
    print("Menguji konfigurasi:", cfg)

    model_try = build_mlp_tuned(
        input_dim=X_tr_tfidf_80_dl.shape[1],
        num_classes=num_classes_v2,
        units_1=cfg["units_1"],
        units_2=cfg["units_2"],
        dropout_1=cfg["dropout_1"],
        dropout_2=cfg["dropout_2"],
        lr=cfg["lr"]
    )

    hist = model_try.fit(
        X_tr_tfidf_80_dl, y_tr_tfidf_80_dl,
        validation_split=0.2,
        epochs=15,
        batch_size=cfg["batch_size"],
        callbacks=[early_stop_v2],
        verbose=0
    )

    max_val_acc = float(np.max(hist.history['val_accuracy']))
    _, test_acc = model_try.evaluate(X_te_tfidf_80_dl, y_te_tfidf_80_dl, verbose=0)

    print(f"val_acc_max={max_val_acc:.4f} | test_acc={test_acc:.4f}")

    if max_val_acc > best_dl_tuned_val_acc:
        best_dl_tuned_val_acc = max_val_acc
        best_dl_tuned_test_acc = float(test_acc)
        best_dl_tuned_cfg = cfg
        best_dl_tuned_model = model_try

print("\nKonfigurasi terbaik deep tuning:", best_dl_tuned_cfg)
print(f"Akurasi validasi terbaik: {best_dl_tuned_val_acc:.4f}")
print(f"Akurasi test deep tuned: {best_dl_tuned_test_acc:.4f}")

Menguji konfigurasi: {'units_1': 256, 'units_2': 128, 'dropout_1': 0.3, 'dropout_2': 0.2, 'lr': 0.001, 'batch_size': 128}
val_acc_max=0.9222 | test_acc=0.8983
Menguji konfigurasi: {'units_1': 384, 'units_2': 192, 'dropout_1': 0.35, 'dropout_2': 0.25, 'lr': 0.0008, 'batch_size': 128}
val_acc_max=0.8949 | test_acc=0.8953
Menguji konfigurasi: {'units_1': 512, 'units_2': 256, 'dropout_1': 0.4, 'dropout_2': 0.3, 'lr': 0.0007, 'batch_size': 256}
val_acc_max=0.8926 | test_acc=0.8924

Konfigurasi terbaik deep tuning: {'units_1': 256, 'units_2': 128, 'dropout_1': 0.3, 'dropout_2': 0.2, 'lr': 0.001, 'batch_size': 128}
Akurasi validasi terbaik: 0.9222
Akurasi test deep tuned: 0.8983


### Inference / Testing (Bukti Output) — Model Klasik dan Deep Learning

Jalankan sel ini untuk menampilkan **output kelas kategorikal** (`negatif`, `netral`, `positif`) dari model klasik terbaik dan model deep learning terbaik.

In [51]:
# Pilih model final otomatis: klasik jika cocok, jika tidak maka deep tuned
classic_threshold = 0.85
use_classic = (best_classic_acc >= classic_threshold) and (best_classic_acc >= best_dl_tuned_test_acc)

if use_classic:
    final_model_type = "classic"
    final_model_name = best_classic_name
else:
    final_model_type = "deep"
    final_model_name = "DL-1 MLP + TF-IDF (80/20) tuned"

print(f"Model klasik terbaik: {best_classic_name} | acc={best_classic_acc:.4f}")
print(f"Model deep tuned terbaik: DL-1 tuned | acc={best_dl_tuned_test_acc:.4f}")
print(f"Model final dipakai: {final_model_name}")

def preprocess_text_for_inference(text):
    return str(text).lower().strip()

def transform_for_word2vec(texts):
    toks = [simple_tokenize(t) for t in texts]
    return np.array([sent_vector(t, w2v_model, 100) for t in toks])

def predict_with_classic(texts):
    texts_clean = [preprocess_text_for_inference(t) for t in texts]

    if best_classic_kind == "svm_tfidf80":
        X_inf = tfidf_80.transform(texts_clean)
    elif best_classic_kind == "rf_w2v80":
        X_inf = transform_for_word2vec(texts_clean)
    else:
        X_inf = tfidf_70.transform(texts_clean)

    pred_code = best_classic_model.predict(X_inf)
    return encoder_v2.inverse_transform(pred_code)

def predict_with_deep_tuned(texts):
    texts_clean = [preprocess_text_for_inference(t) for t in texts]
    X_inf = tfidf_80.transform(texts_clean).astype('float32').toarray()
    probs = best_dl_tuned_model.predict(X_inf, verbose=0)
    pred_code = np.argmax(probs, axis=1)
    return encoder_v2.inverse_transform(pred_code)

# Contoh inference (bukti output di notebook)
sample_texts = [
    "produk standar saja, tidak buruk dan tidak istimewa"
]

pred_classic = predict_with_classic(sample_texts)
pred_deep_tuned = predict_with_deep_tuned(sample_texts)

if final_model_type == "classic":
    pred_final = pred_classic
else:
    pred_final = pred_deep_tuned

inference_df = pd.DataFrame({
    "Teks": sample_texts,
    "Prediksi Klasik": pred_classic,
    "Prediksi Deep Tuned": pred_deep_tuned,
    "Prediksi Final Dipakai": pred_final
})

print("Output inference model klasik, deep tuned, dan final:")
display(inference_df)

Model klasik terbaik: SVM + TF-IDF (80/20) | acc=0.8543
Model deep tuned terbaik: DL-1 tuned | acc=0.8983
Model final dipakai: DL-1 MLP + TF-IDF (80/20) tuned
Output inference model klasik, deep tuned, dan final:


,Teks,Prediksi Klasik,Prediksi Deep Tuned,Prediksi Final Dipakai
0,"produk standar saja, tidak buruk dan tidak ist...",negative,negative,negative
